In [8]:
import random
import jsonlines
import os
from pathlib import Path

In [27]:
contents = []
data_path = Path(os.getcwd()).parent.parent / "data" 
with (data_path / "raw_data" / "agenda" / "agenda_events.jsonl").open('r') as f:
    contents.extend(iter(jsonlines.Reader(f)))

In [28]:
import pandas as pd
df = pd.DataFrame(contents)

In [29]:
print(df.head())

                      event start_time  end_time                location  \
0         Breakfast meeting    9:00 AM  10:00 AM           Bluebird Cafe   
1  Sales pitch presentation   10:30 AM  11:30 AM          Hilton Chicago   
2             Cooking class   11:00 AM   1:00 PM           Sur La Table    
3     Lunch with colleagues    1:00 PM   2:00 PM  The Cheesecake Factory   
4        Charity fundraiser    2:30 PM   4:30 PM        The Ritz-Carlton   

    person        date           id  
0      Mia  2022/12/15  agenda-0000  
1  William  2022/07/05  agenda-0001  
2    Emily  2022/03/04  agenda-0002  
3     Adam  2022/12/02  agenda-0003  
4    Layla  2022/08/21  agenda-0004  


In [30]:
from datetime import datetime

def compute_duration(start_time: str, end_time: str, time_format: str = "%I:%M %p") -> str:
    start_time_obj = datetime.strptime(start_time, time_format)
    end_time_obj = datetime.strptime(end_time, time_format)

    duration = end_time_obj - start_time_obj

    return str(duration)


start_time = df.iloc[0]["start_time"]
end_time = df.iloc[0]["end_time"]
print(compute_duration(start_time, end_time))

1:00:00


In [32]:
random_indices = random.sample(range(len(contents)), 1000)
questions = []
question_id = 0
for index in random_indices:
    row = df.iloc[index]
    content = row
    question_indices = random.randint(0, 4)
    date = content["date"]
    if question_indices == 0:
        # What did {person} do from {start_time} to {end_time} on {date}?
        person = content["person"]
        start_time = content["start_time"]
        end_time = content["end_time"]
        question = f"What did {person} do from {start_time} to {end_time} on {date}?"
        answer = content["event"]
    elif question_indices == 1:
        # Where did {event} that {person} attended take place on {date}?
        person = content["person"]
        event = content["event"]
        question = f"Where did {event} that {person} attended take place on {date}?"
        answer = content["location"]
    elif question_indices == 2:
        # When did {person} attend {event} on {date}?
        person = content["person"]
        event = content["event"]
        question = f"When did {person} attend {event} on {date}?"
        answer = content["start_time"]
    elif question_indices == 3:
        # How long did {person} attend {event} on {date}?
        person = content["person"]
        event = content["event"]
        start_time = content["start_time"]
        end_time = content["end_time"]
        question = f"How long did {person} attend {event} on {date}?"
        answer = compute_duration(start_time, end_time)
    else:
        # Who attended {event} between {start_time} and {end_time} on {date} in {location}?
        event = content["event"]
        start_time = content["start_time"]
        end_time = content["end_time"]
        location = content["location"]
        question = f"Who attended {event} between {start_time} and {end_time} on {date} in {location}?"
        answer = content["person"]

    questions.append(
        {
            "oid": content["id"],
            "qid": "easy-agenda-{:0>4d}".format(question_id),
            "question": question,
            "answer": answer,
        }
    )
    question_id += 1
    if question_id >= 100:
        break

In [33]:
with jsonlines.open(data_path / "questions" / "easy" / "agenda-easy.jsonl", mode='w') as writer:
    for row in questions:
        writer.write(row)